# Trying out Minigrad

In [2]:
from minigrad.engine import Value
from minigrad.nn import NeuralNetwork

In [6]:
#Create a minimal neural network: 2 inputs -> 1 output
nn = NeuralNetwork(2, [2, 1], activation=lambda x: x.tanh())

    # Single training example
X = [[2.0, 0.0]]  # Just one input
y = [1.0]        # Target output

    # Set optimizer
nn.set_optimizer('sgd', lr=0.1)

    # Single forward and backward pass
x = [Value(x) for x in X[0]]
    
    # Forward pass
pred = nn.forward(x)
loss = (pred - Value(y[0]))**2
nn.last_output = loss
    
    # Backward pass
nn.zero_grad()
loss.backward()
    
    # Visualize before optimization step
print(f'Prediction: {pred.data:.4f}')
print(f'Loss: {loss.data:.4f}')
nn_image = nn.draw_nn(loss)


Prediction: 0.2601
Loss: 0.5474
